# Quickstart

With a valid Python environment, nested-dask and its dependencies are easy to install using the `pip` package manager. The following command can be used to install it:

In [ ]:
# % pip install nested-dask

Nested-Dask is a package that enables parallelized computation of nested associated datasets.

Usage of Nested-Dask very closely follows the usage of [Nested-Pandas](https://nested-pandas.readthedocs.io/en/latest/index.html), but with a layer of [Dask](https://www.dask.org/get-started) concepts introduced on top. This quickstart guide will step through a basic example that mirrors the [quickstart guide](https://nested-pandas.readthedocs.io/en/latest/gettingstarted/quickstart.html) of Nested-Pandas. First, let's load some toy data:

In [ ]:
from nested_dask.datasets import generate_data

# generate_data creates some toy data
ndf = generate_data(10, 100)  # 10 rows, 100 nested rows per row
ndf

The above is a Nested-Dask `NestedFrame` object. It's currently a "lazy" representation of the data, meaning that no data has actually been brought into memory yet. This lazy view gives us some useful information on the structure of the data, with notable pieces of information being:

* Shows us which columns are in the dataset and their respective dtypes.

* `npartitions=1` indicates how many partitions the dataset has been split into.

*  The `0` and `9` tell us the "divisions" of the partitions. When the dataset is sorted by the index, these divisions are ranges to show which index values reside in each partition.

We can use peek at the first `n` rows using `ndf.head(n)` (or the last few with `ndf.tail(n)`).

In [ ]:
ndf.head(3)

We can signal to Dask that we'd like to actually obtain *all* of the data as `nested_pandas.NestedFrame` by using `compute`.

In [ ]:
ndf.compute()

As with Nested-Pandas, this `NestedFrame` holds special nested columns in addition to normal Pandas columns. In this case, we have the top level dataframe with 10 rows and 2 typical columns, “a” and “b”. The “nested” column contains a dataframe in each row. We can inspect the contents of the “nested” column using the standard Pandas/Dask API.

In [ ]:
ndf.nested.compute()[0]

Here we see that within the “nested” column there are Nested-Pandas `NestedFrame` objects with their own data. In this case we have 3 columns (“t”, “flux”, and “band”).

Nested-Dask functionality mirrors Nested-Pandas, as we can see via the `query` function. In this case, we use a Nested-Pandas specific feature to query nested layers using a hierarchical column name ("nested.t" queries the "t" sub-column from the "nested" column of ndf).

In [ ]:
# Applies the query to "nested", filtering based on "t >17"
result = ndf.query("nested.t > 17.0")
result

Once again, the result is lazy and no work has been performed. We can kick off some computation using `compute` as above or this time using `head` to just peek at the result:

In [ ]:
result.head(5)

In this case, the query has actually affected the rows of the "nested" column.

In [ ]:
result.head(5).nested[0]  # no `t` value is lower than 17.0

Nested-Dask `reduce` functions near-identically to Nested-Pandas `reduce`, providing a way to call custom functions on `NestedFrame` data. The one additional concern is that Dask requires, in almost every case, a `meta=` argument to help Dask understand the shape and type of the output data. Dask provides a `make_meta` function, to which you can pass a dummy output value.

In [ ]:
import numpy as np
import pandas as pd
from dask.dataframe.utils import make_meta

# Use hierarchical column names to access the flux column
# passed as an array to np.mean .
#
# Take a single sample row, computed (that's what .head(1) will do),
# and generate the meta for it.
meta = make_meta(ndf.head(1).reduce(np.mean, "nested.flux"))

means = ndf.reduce(np.mean, "nested.flux", meta=meta)
means.compute()

The `reduce` function can also be used to apply any row-based calculation, as it turns out, even if the dimension stays the same. Observe that we can use this similar pattern to produce, say, the square of the flux. It is still a "reduction" in that the result is no longer within the original `NestedFrame` structure, but the cardinality of each output row is now the same as the cardinality of each input row.

In [ ]:
meta = make_meta(ndf.head(1).reduce(np.square, "nested.flux"))

flux_sq = ndf.reduce(np.square, "nested.flux", meta=meta)
flux_sq.compute()